In [35]:
import numpy as np
from keras.datasets import mnist  
from keras.utils import to_categorical

class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size, learning_rate):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.learning_rate = learning_rate
        
        # Initialize weights and biases
        self.W1 = np.random.randn(input_size, hidden_size)
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = np.random.randn(hidden_size, output_size)
        self.b2 = np.zeros((1, output_size))

    #Activation Functions
    def relu(self, x):
        return np.maximum(0, x)

    def softmax(self, x):
        exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)
    
    #Forward Propagation
    def forward(self, X):
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = self.relu(self.z1)
        self.z2 = np.dot(self.a1, self.W2) + self.b2
        self.a2 = self.softmax(self.z2)
        return self.a2
    
    #Backward Propagation
    def backward(self, X, y):
        m = X.shape[0]
        
        # Compute gradients
        dz2 = self.a2 - y
        dW2 = (1 / m) * np.dot(self.a1.T, dz2)
        db2 = (1 / m) * np.sum(dz2, axis=0)
        dz1 = np.dot(dz2, self.W2.T) * (self.z1 > 0)  # ReLU gradient
        dW1 = (1 / m) * np.dot(X.T, dz1)
        db1 = (1 / m) * np.sum(dz1, axis=0)
        
        # Update weights and biases
        self.W2 -= self.learning_rate * dW2
        self.b2 -= self.learning_rate * db2
        self.W1 -= self.learning_rate * dW1
        self.b1 -= self.learning_rate * db1

    def Accuracy(self,output,y):
        # Calculate accuracy
        predictions = np.argmax(output, axis=1)
        true_labels = np.argmax(y, axis=1)
        accurate = np.mean(predictions == true_labels)
        return accurate
    
    #Training Function
    def train(self, X, y, epochs):
        for epoch in range(epochs):
            # Forward and backward pass
            output = self.forward(X)
            self.backward(X, y)
            output=self.forward(X)
            loss=np.mean((y-output)**2)
            accuracy=self.Accuracy(output,y)
            if epoch%100==0:
                print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss:.4f}, Accuracy: {accuracy * 100:.2f}%")
    
    #Testing Function        
    def test(self,X,y):
        output=self.forward(X)
        correct = 0
        total = len(X)

        for i in range(total):
            prediction = np.argmax(self.forward(X[i]))
            true_label = np.argmax(y[i])
            if prediction == true_label:
                correct += 1

        accuracy = (correct / total) * 100
        print(f"Accuracy on test data: {accuracy:.2f}%")


In [36]:
if __name__ == "__main__":
    # Load MNIST dataset
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    
    # Flatten and normalize the data
    X_train = X_train.reshape(60000, 28 * 28) / 255.0
    X_test = X_test.reshape(10000, 28 * 28) / 255.0
    
    # One-hot encode labels
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    
    # Initialize and train the neural network
    input_size = 28*28
    hidden_size = 256
    output_size = 10
    learning_rate = 0.3
    
    #Neural Network Layer
    nn = NeuralNetwork(input_size, hidden_size, output_size, learning_rate)
    nn.train(X_train, y_train, epochs=1000)
    nn.test(X_test,y_test)

Epoch 1/1000, Loss: 0.1463, Accuracy: 26.11%
Epoch 101/1000, Loss: 0.0239, Accuracy: 87.32%
Epoch 201/1000, Loss: 0.0186, Accuracy: 89.94%
Epoch 301/1000, Loss: 0.0158, Accuracy: 91.34%
Epoch 401/1000, Loss: 0.0139, Accuracy: 92.25%
Epoch 501/1000, Loss: 0.0125, Accuracy: 92.97%
Epoch 601/1000, Loss: 0.0114, Accuracy: 93.51%
Epoch 701/1000, Loss: 0.0106, Accuracy: 93.89%
Epoch 801/1000, Loss: 0.0098, Accuracy: 94.30%
Epoch 901/1000, Loss: 0.0092, Accuracy: 94.61%
Accuracy on test data: 92.53%
